In [1]:
import pandas as pd
from sklearn.utils import resample
df = pd.read_csv('train.csv')

In [8]:
def preprocessing_optimal(df):
    df = df.drop(["_id", "age", 'job', "marital", "education", "default",
                 "housing", "loan", "day_of_week", 'campaign', 'previous', 'emp.var.rate', "cons.price.idx",
                 "cons.conf.idx", "euribor3m", 'pdays'], axis = 1)
    df = pd.get_dummies(df, columns=['contact', 'month', 'poutcome'])
    return df
def preprocessing(df_input, algorithm = preprocessing_optimal):
    return algorithm(df_input)

In [9]:
from sklearn.preprocessing import StandardScaler
#обрабатываем данные инициализируем X и Y
df_prepared = preprocessing(df)
Y = df_prepared['target']
X = StandardScaler().fit_transform(df_prepared.drop('target', axis=1))

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.7, random_state=12134)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(17298, 17) (7414, 17) (17298,) (7414,)


C:\Users\Evgeny\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score

In [35]:
logreg = LogisticRegression().fit(X_train, y_train)
decision_tree = DecisionTreeClassifier(max_depth = 5).fit(X_train, y_train)
random_forest = RandomForestClassifier(max_depth = 8).fit(X_train, y_train)
knn = KNeighborsClassifier(n_neighbors=20).fit(X_train, y_train)

In [46]:
model = random_forest
lst = list()
for i in range (1,21):
    lst.append(roc_auc_score(y_test, 
        [ 1 if y > (0.02 * i) else 0 for y in model.predict_proba(X_test)[:,1]]))
max(lst)

0.8739181237161758